In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import f_oneway
import math
from numpy import array
import plotly.graph_objects as go

In [31]:
# UTILS

# Розрахунок MA
def moving_average(data, window_size):
    ma = []
    for i in range(len(data) - window_size + 1):
        window = data[i:i + window_size]
        ma.append(sum(window) / window_size)
    return ma

# Розрахунок RSI
def get_rsi(close, lookback):
    ret = close.diff()
    up = []
    down = []
    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    up_ewm = up_series.ewm(com = lookback - 1, adjust = False).mean()
    down_ewm = down_series.ewm(com = lookback - 1, adjust = False).mean()
    rs = up_ewm/down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns = {0:'rsi'}).set_index(close.index)
    rsi_df = rsi_df.dropna()
    return rsi_df[3:]

# Перевірка даних на пропушенні значення
def checkMissVal(data):
  missing_values = data.isnull().sum()
  print("Counts of missing values:")
  print(missing_values) 

# Перевірка даних на викиди за допомогою методу Local Outlier Factor (LOF)
def detect_outliers_lof(data, n_neighbors=20, contamination=0.1):
    lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
    outliers = lof.fit_predict(data)
    data['LOF'] = outliers
    
    return data[data['LOF'] == -1]
  
# Описові статистики
def descriptive_statistics(column):
    stats = column.describe()
    
    print("Описові статистики для стовпчика:")
    print(stats)

# Створення графіку
def plot_line_chart(dataframe, title, yaxis_title, y_columns=None, x_column=None):
    fig = go.Figure()
    
    columns = y_columns if y_columns else dataframe.columns
    xaxis = dataframe[x_column] if x_column in dataframe.columns else dataframe.index

    for column in columns:
        fig.add_trace(go.Scatter(x=xaxis, y=dataframe[column], mode='lines', name=column))

    fig.update_layout(title=title, xaxis_title='Дата', yaxis_title=yaxis_title)

    fig.show()

In [45]:
# Першим етапом нашого дослідження дістанемо наші дані та візуалізуємо їх

data = pd.read_csv('BTC-USD.csv')

print(data.head())

plot_line_chart(data, "Біржові дані по BTC за рік", "$", y_columns=["Open", "High", "Low", "Close"], x_column="Data")

       Date         Open         High          Low        Close        Volume
0  4/1/2023  28473.33203  28802.45703  28297.17188  28411.03516  1.087647e+10
1  4/2/2023  28462.84570  28518.95898  27884.08789  28199.30859  1.228464e+10
2  4/3/2023  28183.08008  28475.62305  27276.72070  27790.22070  1.955650e+10
3  4/4/2023  27795.27344  28433.74219  27681.30469  28168.08984  1.528454e+10
4  4/5/2023  28169.72656  28739.23828  27843.76367  28177.98438  1.705232e+10


In [46]:
# Розрахунок MA

# Розрахунок MA
window_size = 20  # Розмір вікна для MA
data['MA'] = data['Close'].rolling(window=window_size).mean()

# Виведення перших кількох рядків DataFrame для перевірки результату
display(data.head())

,Date,Open,High,Low,Close,Volume,MA
0,4/1/2023,28473.33203,28802.45703,28297.17188,28411.03516,1.087647e+10,NaN
1,4/2/2023,28462.84570,28518.95898,27884.08789,28199.30859,1.228464e+10,NaN
2,4/3/2023,28183.08008,28475.62305,27276.72070,27790.22070,1.955650e+10,NaN
3,4/4/2023,27795.27344,28433.74219,27681.30469,28168.08984,1.528454e+10,NaN
4,4/5/2023,28169.72656,28739.23828,27843.76367,28177.98438,1.705232e+10,NaN


In [35]:
# Розрахунок параметру MACD

# Параметри
short_window = 14
long_window = 28
signal_window = 7

# Розрахунок експоненційних середніх (EMA)
data['EMA_short'] = data['Close'].ewm(span=short_window, adjust=False).mean()
data['EMA_long'] = data['Close'].ewm(span=long_window, adjust=False).mean()

# Розрахунок MACD
data['MACD'] = data['EMA_short'] - data['EMA_long']

# Розрахунок сигнальної лінії
data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()

# Виведення перших кількох рядків DataFrame для перевірки результату
print(data.head())

       Date         Open         High          Low        Close        Volume  \
0  4/1/2023  28473.33203  28802.45703  28297.17188  28411.03516  1.087647e+10   
1  4/2/2023  28462.84570  28518.95898  27884.08789  28199.30859  1.228464e+10   
2  4/3/2023  28183.08008  28475.62305  27276.72070  27790.22070  1.955650e+10   
3  4/4/2023  27795.27344  28433.74219  27681.30469  28168.08984  1.528454e+10   
4  4/5/2023  28169.72656  28739.23828  27843.76367  28177.98438  1.705232e+10   

   MA     EMA_short      EMA_long       MACD  Signal_Line  
0 NaN  28411.035160  28411.035160   0.000000     0.000000  
1 NaN  28382.804951  28396.433328 -13.628377    -3.407094  
2 NaN  28303.793717  28354.625560 -50.831843   -15.263281  
3 NaN  28285.699867  28341.761028 -56.061161   -25.462751  
4 NaN  28271.337802  28330.466087 -59.128284   -33.879135  


In [36]:
# Розраховуваємо RSI
rsi_values = get_rsi(data["Close"], lookback=14)

data['RSI'] = rsi_values

In [37]:
# Трансофрмація датасету, видаляємо з нього NaN

data = data.dropna()
data.reset_index(drop=True, inplace=True)
data.index = data["Date"]
data = data.drop("Date", axis=1)
data.tail()



,Open,High,Low,Close,Volume,MA,EMA_short,EMA_long,MACD,Signal_Line,RSI
Date,,,,,,,,,,,
5/7/2024,63162.76172,64390.45703,62285.98047,62334.81641,2.593073e+10,63386.664454,62963.152031,63860.294311,-897.142280,-988.268116,45.781929
5/8/2024,62332.64063,62986.08594,60877.12891,61187.94141,2.608817e+10,63270.423829,62726.457281,63675.994111,-949.536830,-978.585295,42.973872
5/9/2024,61191.19922,63404.91406,60648.07422,63049.96094,2.545334e+10,63230.743361,62769.591102,63632.819410,-863.228307,-949.746048,48.497139
5/10/2024,63055.19141,63446.74219,60208.78125,60792.77734,2.780495e+10,63020.660157,62506.015934,63436.954439,-930.938505,-945.044162,43.053404
5/11/2024,60791.75781,61059.21094,60708.19531,61059.21094,2.696748e+10,62827.288478,62313.108602,63272.972129,-959.863527,-948.749003,43.854532


In [38]:
plot_line_chart(data, "Біржові дані з розрахованими параметрами MA та EMA", "$", y_columns=["Close", "MA", "EMA_short", "EMA_long"], x_column="Data")

На графіку цін та їх скользящих середніх, таких як проста скользяща середня (MA) та експоненційна скользяща середня (EMA_short та EMA_long), спостерігається загальна тенденція до зниження цін на Bitcoin на ринку. Ці показники є важливими інструментами для аналізу цінових рухів, оскільки дозволяють виявити загальний напрямок цін, відфільтрувати випадкові коливання та визначити стабільні тренди відповідно до певного періоду часу. Залежно від їхньої взаємодії можна зробити висновки щодо майбутніх рухів цін та прийняти відповідні торгівельні рішення. Проте варто зауважити, що криптовалюти можуть бути непередбачуваними та змінювати свою тенденцію раптово, тому точний прогноз майбутніх рухів не завжди можливий. Треба побудвати додткові індикатори.

In [42]:
macd = data["MACD"]
signal_line  = data["Signal_Line"]

# Побудова графіку
trace_macd = go.Scatter(x=data.index, y=macd, mode='lines', name='MACD')
trace_signal_line = go.Scatter(x=data.index, y=signal_line, mode='lines', name='Signal Line')

# Розрахунок гістограми сигналу
histogram = macd - signal_line
trace_histogram = go.Bar(x=data.index, y=histogram, name='Histogram', marker=dict(color='rgba(0, 0, 255, 0.7)'))

# Створення графіку
layout = go.Layout(title='MACD with Signal Line and Histogram', xaxis=dict(title='Date'), yaxis=dict(title='MACD'))

fig = go.Figure(data=[trace_macd, trace_signal_line, trace_histogram], layout=layout)

# Відображення графіку
fig.show()

На початку місяця MACD був нижче за лінію сигналу, що може вказувати на переважання від'ємного настрою на ринку. Це може свідчити про загальну негативну тенденцію або зниження цін.

У середині місяця MACD перетнув гістограму зверху вниз та став у від'ємному діапазоні. Це може бути ознакою зміни зростання на зниження і може вказувати на загострення негативного настрою на ринку.

Під кінець місяця MACD трохи перевищив лінію сигналу, але все ще перебував у від'ємному діапазоні. Це може бути ознакою тимчасового збільшення позитивного настрою, але загальна тенденція залишалася негативною.

In [44]:
# Побудова графіка
trace_rsi = go.Scatter(x=data.index, y=data['RSI'], mode='lines', name='RSI', line=dict(color='orange'))

plot_line_chart(data, "Біржові дані", "$", y_columns=["Close"], x_column="Data")

# Граничні лінії
upper_limit = 70
lower_limit = 30
trace_upper_limit = go.Scatter(x=data.index, y=[upper_limit] * len(data), mode='lines', name='Upper Limit', line=dict(color='black', dash='dash'))
trace_lower_limit = go.Scatter(x=data.index, y=[lower_limit] * len(data), mode='lines', name='Lower Limit', line=dict(color='black', dash='dash'))

# Створення графіку
layout = go.Layout(title='RSI with Upper and Lower Limits', xaxis=dict(title='Date'), yaxis=dict(title='RSI'))

fig = go.Figure(data=[trace_rsi, trace_upper_limit, trace_lower_limit], layout=layout)

# Відображення графіку
fig.show()

У контексті аналізу індексу сили ціни (RSI) зафіксовано його значення в діапазоні від 60 до 35. Це вказує на те, що вирослі дні надто помірні в порівнянні з впалими днями протягом останнього періоду. Така динаміка може свідчити про помірний позитивний настрій на ринку, проте, ймовірно, вказує на початок послаблення цього настрою.

Додатково, враховуючи графік цін, слід відзначити наявність спадаючого тренду. Це означає, що ціни показують схильність до поступового зниження протягом аналізованого періоду. Такий тренд може вказувати на загальний негативний настрій на ринку та можливий подальший рух цін вниз.

Узагальнюючи, враховуючи інформацію про RSI та спостереження на графіку цін, можна зробити припущення, що на ринку може змінюватися позитивний настрій на більш нейтральний або навіть негативний, що може відображатися в подальшому зниженні цін.

В результаті застосування декількох методів технічного аналізу ми спостерігаємо, що на поточний момент ринок перебуває у фазі зниження тренду. Це підтверджується всіма розрахованими показниками. Можна припускати, що в найближчому майбутньому тренд може залишатися стійким або помірно спадати, або можливо відбудеться підйом. З метою отримання більш повної карти ми розглянемо методи кількісного аналізу, які допоможуть нам зробити більш точні прогнози щодо майбутніх періодів.